In [14]:
import pandas as pd

EEG_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\eeg_features_FINAL_two_channel_corr.csv"
EMG_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\emg_features_SELECTED_FINAL_after_corr.csv"
ECG_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\ecg_features_SELECTED_FINAL_after_corr.csv"
MOV_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\MOV_features_SELECTED_KW_p005-2.csv"

# Load
eeg = pd.read_csv(EEG_CSV)
emg = pd.read_csv(EMG_CSV)
ecg = pd.read_csv(ECG_CSV)
mov = pd.read_csv(MOV_CSV)

print("==================================")
print("EEG Columns:")
print(eeg.columns.tolist())

print("\n==================================")
print("EMG Columns:")
print(emg.columns.tolist())

print("\n==================================")
print("ECG Columns:")
print(ecg.columns.tolist())

print("\n==================================")
print("MOV Columns:")
print(mov.columns.tolist())


EEG Columns:
['subject', 'run', 'window_idx', 'label', 'ch1_Delta Bandpower', 'ch1_Theta Bandpower', 'ch1_Alpha Bandpower', 'ch1_Beta Bandpower', 'ch1_Gamma Bandpower', 'ch1_Relative Delta Bandpower', 'ch1_Relative Theta Bandpower', 'ch1_Relative Alpha Bandpower', 'ch1_Relative Beta Bandpower', 'ch1_Relative Gamma Bandpower', 'ch1_Interquartile Range', 'ch1_Median', 'ch1_Entropy', 'ch1_Skewness', 'ch1_Kurtosis', 'ch1_Line Length', 'ch1_Hjorth Mobility', 'ch1_Hjorth Complexity', 'ch2_Delta Bandpower', 'ch2_Theta Bandpower', 'ch2_Alpha Bandpower', 'ch2_Beta Bandpower', 'ch2_Gamma Bandpower', 'ch2_Relative Delta Bandpower', 'ch2_Relative Theta Bandpower', 'ch2_Relative Alpha Bandpower', 'ch2_Relative Beta Bandpower', 'ch2_Relative Gamma Bandpower', 'ch2_Interquartile Range', 'ch2_Median', 'ch2_Entropy', 'ch2_Skewness', 'ch2_Kurtosis', 'ch2_Line Length', 'ch2_Hjorth Mobility', 'ch2_Hjorth Complexity']

EMG Columns:
['subject', 'run', 'window_idx', 'label', 'session', 'RMS', 'ZC', 'SSC', 'K

In [15]:
import pandas as pd

# ==========================
# 1) File paths
# ==========================

EEG_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\eeg_features_FINAL_two_channel_corr.csv"
EMG_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\emg_features_SELECTED_FINAL_after_corr.csv"
ECG_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\ecg_features_SELECTED_FINAL_after_corr.csv"
MOV_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\MOV_features_SELECTED_KW_p005-2.csv"

OUT_FUSED_EEG_EMG_ECG = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_EMG_ECG.csv"

# ==========================
# 2) Read CSV files
# ==========================
eeg = pd.read_csv(EEG_CSV)
emg = pd.read_csv(EMG_CSV)
ecg = pd.read_csv(ECG_CSV)
mov = pd.read_csv(MOV_CSV)

print("EEG:", eeg.shape, "EMG:", emg.shape, "ECG:", ecg.shape, "MOV:", mov.shape)

# ==========================
# 3) Normalize keys (EEG/EMG/ECG)
# ==========================
def normalize_keys_eeg(df, name):
    df = df.copy()
    df["subject"] = df["subject"].astype(str).str.strip()
    if "session" in df.columns:
        df["session"] = df["session"].astype(str).str.strip()
    df["run"] = df["run"].astype(int)
    df["window_idx"] = df["window_idx"].astype(int)
    return df

eeg = normalize_keys_eeg(eeg, "EEG")
emg = normalize_keys_eeg(emg, "EMG")
ecg = normalize_keys_eeg(ecg, "ECG")

# MOV: we only need subject to verify modality completeness
mov["subject"] = mov["subject"].astype(str).str.strip()

# ==========================
# 4) Drop subjects missing any modality
# ==========================
subs_eeg = set(eeg["subject"].unique())
subs_emg = set(emg["subject"].unique())
subs_ecg = set(ecg["subject"].unique())
subs_mov = set(mov["subject"].unique())

common_subs = subs_eeg & subs_emg & subs_ecg & subs_mov
print("Number of subjects in all 4 modalities:", len(common_subs))

eeg = eeg[eeg["subject"].isin(common_subs)].copy()
emg = emg[emg["subject"].isin(common_subs)].copy()
ecg = ecg[ecg["subject"].isin(common_subs)].copy()
# MOV only used to filter subjects

# ==========================
# 5) Keep only runs that exist in EEG
# ==========================
run_keys = ["subject", "run"]

eeg_runs = eeg[run_keys].drop_duplicates()

emg = emg.merge(eeg_runs, on=run_keys, how="inner")
ecg = ecg.merge(eeg_runs, on=run_keys, how="inner")

print("After aligning runs with EEG:")
print("EEG:", eeg.shape, "EMG:", emg.shape, "ECG:", ecg.shape)

# ==========================
# 6) Merge keys + metadata
# ==========================
key_cols = run_keys + ["window_idx"]

meta_cols = ["subject", "run", "window_idx", "label"]
meta_cols = [c for c in meta_cols if c in eeg.columns]

# ==========================
# 7) Prepare EMG/ECG: drop metadata + add prefix
# ==========================
def prepare_modality(df, prefix):
    drop_meta = [c for c in ["subject", "session", "run", "window_idx", "label"]
                 if c in df.columns]
    feat_cols = [c for c in df.columns if c not in drop_meta]
    df_feat = df[key_cols + feat_cols].copy()
    df_feat = df_feat.rename(columns={c: f"{prefix}{c}" for c in feat_cols})
    return df_feat

emg_feat = prepare_modality(emg, "emg_")
ecg_feat = prepare_modality(ecg, "ecg_")

# ==========================
# 8) Fuse EEG + EMG + ECG
# ==========================
fused_3 = eeg.merge(emg_feat, on=key_cols, how="left")
fused_3 = fused_3.merge(ecg_feat, on=key_cols, how="left")

print("FUSED_EEG_EMG_ECG shape:", fused_3.shape)
print("Number of EEG rows:", eeg.shape[0])

assert fused_3.shape[0] == eeg.shape[0], "❌ Row count must match EEG exactly!"

fused_3.to_csv(OUT_FUSED_EEG_EMG_ECG, index=False, encoding="utf-8-sig")
print("✅ Saved:", OUT_FUSED_EEG_EMG_ECG)


EEG: (5403559, 40) EMG: (5207665, 13) ECG: (5207665, 15) MOV: (5411097, 30)
Number of subjects in all 4 modalities: 113
After aligning runs with EEG:
EEG: (5103389, 40) EMG: (5020944, 13) ECG: (5020944, 15)
FUSED_EEG_EMG_ECG shape: (5103389, 58)
Number of EEG rows: 5103389
✅ Saved: C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_EMG_ECG.csv


In [16]:
# ==========================
# FUSE EEG + MOV
# ==========================
import pandas as pd

EEG_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\eeg_features_FINAL_two_channel_corr.csv"
EMG_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\emg_features_SELECTED_FINAL_after_corr.csv"
ECG_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\ecg_features_SELECTED_FINAL_after_corr.csv"
MOV_CSV = r"C:\Users\LENOVO\Desktop\sen\sen\FIN_DAT\MOV_features_SELECTED_KW_p005-2.csv"

OUT_FUSED_EEG_MOV = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_MOV.csv"

eeg = pd.read_csv(EEG_CSV)
emg = pd.read_csv(EMG_CSV)
ecg = pd.read_csv(ECG_CSV)
mov = pd.read_csv(MOV_CSV)

print("EEG:", eeg.shape, "EMG:", emg.shape, "ECG:", ecg.shape, "MOV:", mov.shape)

def normalize_keys_eeg_like(df):
    df = df.copy()
    df["subject"] = df["subject"].astype(str).str.strip()
    if "session" in df.columns:
        df["session"] = df["session"].astype(str).str.strip()
    df["run"] = df["run"].astype(int)
    df["window_idx"] = df["window_idx"].astype(int)
    return df

eeg = normalize_keys_eeg_like(eeg)
emg = normalize_keys_eeg_like(emg)
ecg = normalize_keys_eeg_like(ecg)

mov = mov.copy()
mov["subject"] = mov["subject"].astype(str).str.strip()
mov["run"] = mov["run_base"].str.extract(r"run-(\d+)").astype(int)
mov["window_idx"] = mov["win_idx"].astype(int)

print("\nMOV after adding run and window_idx:")
print(mov[["subject", "run_base", "run", "win_idx", "window_idx"]].head())

subs_eeg = set(eeg["subject"].unique())
subs_emg = set(emg["subject"].unique())
subs_ecg = set(ecg["subject"].unique())
subs_mov = set(mov["subject"].unique())

common_subs = subs_eeg & subs_emg & subs_ecg & subs_mov
print("\nNumber of subjects present in all 4 modalities:", len(common_subs))

eeg = eeg[eeg["subject"].isin(common_subs)].copy()
emg = emg[emg["subject"].isin(common_subs)].copy()
ecg = ecg[ecg["subject"].isin(common_subs)].copy()
mov = mov[mov["subject"].isin(common_subs)].copy()

print("After subject filtering:")
print("EEG:", eeg.shape, "EMG:", emg.shape, "ECG:", ecg.shape, "MOV:", mov.shape)

run_keys = ["subject", "run"]
eeg_runs = eeg[run_keys].drop_duplicates()

emg = emg.merge(eeg_runs, on=run_keys, how="inner")
ecg = ecg.merge(eeg_runs, on=run_keys, how="inner")
mov = mov.merge(eeg_runs, on=run_keys, how="inner")

print("\nAfter aligning runs with EEG:")
print("EEG:", eeg.shape, "EMG:", emg.shape, "ECG:", ecg.shape, "MOV:", mov.shape)

key_cols = run_keys + ["window_idx"]

meta_cols = ["subject", "run", "window_idx", "label"]
meta_cols = [c for c in meta_cols if c in eeg.columns]
eeg_meta = eeg[meta_cols].copy()

def prepare_mov_no_prefix(df):
    drop_meta = [c for c in ["subject", "session", "run", "window_idx", "label"]
                 if c in df.columns]
    feat_cols = [c for c in df.columns if c not in drop_meta]
    df_feat = df[key_cols + feat_cols].copy()
    return df_feat

mov_feat = prepare_mov_no_prefix(mov)

print("\nExample MOV feature columns (no prefix):")
print([c for c in mov_feat.columns if c not in key_cols][:10])

fused_mov = eeg_meta.merge(mov_feat, on=key_cols, how="left")

print("\nFUSED_EEG_MOV shape:", fused_mov.shape)
print("EEG rows after filtering:", eeg_meta.shape[0])

assert fused_mov.shape[0] == eeg_meta.shape[0], "❌ Row count must match EEG exactly!"

fused_mov.to_csv(OUT_FUSED_EEG_MOV, index=False, encoding="utf-8-sig")
print("\n✅ Saved FUSED_EEG_MOV to:")
print(OUT_FUSED_EEG_MOV)


EEG: (5403559, 40) EMG: (5207665, 13) ECG: (5207665, 15) MOV: (5411097, 30)

MOV after adding run and window_idx:
   subject                                 run_base  run  win_idx  window_idx
0  sub-001  sub-001_ses-01_task-szMonitoring_run-03    3        0           0
1  sub-001  sub-001_ses-01_task-szMonitoring_run-03    3        1           1
2  sub-001  sub-001_ses-01_task-szMonitoring_run-03    3        2           2
3  sub-001  sub-001_ses-01_task-szMonitoring_run-03    3        3           3
4  sub-001  sub-001_ses-01_task-szMonitoring_run-03    3        4           4

Number of subjects present in all 4 modalities: 113
After subject filtering:
EEG: (5103389, 40) EMG: (5129336, 13) ECG: (5129336, 15) MOV: (5211781, 32)

After aligning runs with EEG:
EEG: (5103389, 40) EMG: (5020944, 13) ECG: (5020944, 15) MOV: (5103389, 32)

Example MOV feature columns (no prefix):
['Unnamed: 0', 'win_idx', 't_start', 't_end', 'timestamp_center', 'EEG_ACC_MAG_std', 'EEG_ACC_MAG_energy', 'EEG_ACC

In [17]:
import pandas as pd

FUSED_3   = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_EMG_ECG.csv"
FUSED_MOV = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_MOV.csv"

f3  = pd.read_csv(FUSED_3)
fm  = pd.read_csv(FUSED_MOV)

print("Rows FUSED_3   :", f3.shape[0])
print("Rows FUSED_MOV :", fm.shape[0])

key_cols = ["subject", "run", "window_idx"]

for name, df in [("FUSED_3", f3), ("FUSED_MOV", fm)]:
    missing = [c for c in key_cols if c not in df.columns]
    if missing:
        print(f"❌ {name} is missing columns: {missing}")
    else:
        print(f"✅ {name} has merge keys:", key_cols)

def key_set(df):
    return set(zip(df["subject"], df["run"], df["window_idx"]))

f3_keys = key_set(f3)
fm_keys = key_set(fm)

print("\nNumber of unique windows (subject, run, window_idx):")
print("FUSED_3   :", len(f3_keys))
print("FUSED_MOV :", len(fm_keys))

if f3.shape[0] == fm.shape[0] and f3_keys == fm_keys:
    print("\n✅ Both files are built on exactly the same windows.")
else:
    print("\n⚠ There is a mismatch between the two files.")


Rows FUSED_3   : 5103389
Rows FUSED_MOV : 5103389
✅ FUSED_3 has merge keys: ['subject', 'run', 'window_idx']
✅ FUSED_MOV has merge keys: ['subject', 'run', 'window_idx']

Number of unique windows (subject, run, window_idx):
FUSED_3   : 5103389
FUSED_MOV : 5103389

✅ Both files are built on exactly the same windows.


In [18]:
import pandas as pd

FUSED_3   = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_EMG_ECG.csv"
FUSED_MOV = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_MOV.csv"

# Load both
df3 = pd.read_csv(FUSED_3)
dfm = pd.read_csv(FUSED_MOV)

print("\n==============================")
print("FUSED EEG+EMG+ECG Columns:")
print(df3.columns.tolist())
print("Number of columns:", len(df3.columns))

print("\n==============================")
print("FUSED EEG+MOV Columns:")
print(dfm.columns.tolist())
print("Number of columns:", len(dfm.columns))



FUSED EEG+EMG+ECG Columns:
['subject', 'run', 'window_idx', 'label', 'ch1_Delta Bandpower', 'ch1_Theta Bandpower', 'ch1_Alpha Bandpower', 'ch1_Beta Bandpower', 'ch1_Gamma Bandpower', 'ch1_Relative Delta Bandpower', 'ch1_Relative Theta Bandpower', 'ch1_Relative Alpha Bandpower', 'ch1_Relative Beta Bandpower', 'ch1_Relative Gamma Bandpower', 'ch1_Interquartile Range', 'ch1_Median', 'ch1_Entropy', 'ch1_Skewness', 'ch1_Kurtosis', 'ch1_Line Length', 'ch1_Hjorth Mobility', 'ch1_Hjorth Complexity', 'ch2_Delta Bandpower', 'ch2_Theta Bandpower', 'ch2_Alpha Bandpower', 'ch2_Beta Bandpower', 'ch2_Gamma Bandpower', 'ch2_Relative Delta Bandpower', 'ch2_Relative Theta Bandpower', 'ch2_Relative Alpha Bandpower', 'ch2_Relative Beta Bandpower', 'ch2_Relative Gamma Bandpower', 'ch2_Interquartile Range', 'ch2_Median', 'ch2_Entropy', 'ch2_Skewness', 'ch2_Kurtosis', 'ch2_Line Length', 'ch2_Hjorth Mobility', 'ch2_Hjorth Complexity', 'emg_RMS', 'emg_ZC', 'emg_SSC', 'emg_KURTOSIS', 'emg_SKEWNESS', 'emg_MEAN_

In [22]:
import pandas as pd

path = r"C:\Users\LENOVO\Desktop\sen\sen\Data_ALL.csv"

df = pd.read_csv(path)

print("Number of rows:", len(df))
print("Number of columns:", len(df.columns))
print("Columns:", df.columns.tolist()[:15], "...")


Number of rows: 1622677
Number of columns: 113
Columns: ['subject', 'run', 'window_idx', 'label', 'ch1_Delta Bandpower', 'ch1_Theta Bandpower', 'ch1_Alpha Bandpower', 'ch1_Beta Bandpower', 'ch1_Gamma Bandpower', 'ch1_Relative Delta Bandpower', 'ch1_Relative Theta Bandpower', 'ch1_Relative Alpha Bandpower', 'ch1_Relative Beta Bandpower', 'ch1_Relative Gamma Bandpower', 'ch1_Interquartile Range'] ...


In [23]:
import pandas as pd

# ==========================
# 1) Paths
# ==========================
DATA_ALL_PATH   = r"C:\Users\LENOVO\Desktop\sen\sen\Data_ALL.csv"

FUSED_3_IN      = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_EMG_ECG_FIXED.csv"
FUSED_MOV_IN    = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_MOV_FIXED.csv"

FUSED_3_OUT     = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_EMG_ECG_FINAL_FROM_DATA_ALL.csv"
FUSED_MOV_OUT   = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_MOV_FINAL_FROM_DATA_ALL.csv"

# ==========================
# 2) Load Data_ALL (master meta + labels)
# ==========================
print("Loading Data_ALL ...")
df_all = pd.read_csv(DATA_ALL_PATH)

# Normalize key types
key_cols = ["subject", "run", "window_idx"]

df_all["subject"]    = df_all["subject"].astype(str).str.strip()
df_all["run"]        = pd.to_numeric(df_all["run"], errors="raise").astype(int)
df_all["window_idx"] = pd.to_numeric(df_all["window_idx"], errors="raise").astype(int)
df_all["label"]      = pd.to_numeric(df_all["label"], errors="coerce").fillna(0).astype(int)

meta_all = df_all[key_cols + ["label"]].rename(columns={"label": "label_all"})

print("Data_ALL shape:", df_all.shape)
print("Unique (subject,run,window_idx) in Data_ALL:", meta_all[key_cols].drop_duplicates().shape[0])


# ==========================
# 3) Helper to align + replace labels
# ==========================
def align_and_replace_labels(fused_in, fused_out, meta_all, key_cols):
    print("\n====================================")
    print(f"Processing fused file: {fused_in}")
    df = pd.read_csv(fused_in)
    print("Shape (before):", df.shape)

    # Normalize key + label types
    df["subject"]    = df["subject"].astype(str).str.strip()
    df["run"]        = pd.to_numeric(df["run"], errors="raise").astype(int)
    df["window_idx"] = pd.to_numeric(df["window_idx"], errors="raise").astype(int)
    df["label"]      = pd.to_numeric(df["label"], errors="coerce").fillna(0).astype(int)

    # Keep copy of original label for sanity check
    df["label_orig"] = df["label"].copy()

    # Inner-join with Data_ALL meta:
    # - This drops any rows NOT present in Data_ALL
    # - Brings in label_all (authoritative label)
    merged = df.merge(
        meta_all,
        on=key_cols,
        how="inner",
        validate="m:1"   # each (sub,run,win) in fused should match at most 1 in Data_ALL
    )

    print("Shape after inner-join with Data_ALL:", merged.shape)

    # Compare old vs new labels (just for info)
    mismatch_mask = merged["label_orig"] != merged["label_all"]
    n_mismatch = int(mismatch_mask.sum())
    print(f"Number of rows where label_orig != label_all: {n_mismatch}")

    # Replace label with label_all
    merged = merged.drop(columns=["label"])
    merged = merged.rename(columns={"label_all": "label"})

    # If you don't want to keep label_orig, drop it (optional)
    merged = merged.drop(columns=["label_orig"])

    # Save final fused file
    merged.to_csv(fused_out, index=False, encoding="utf-8-sig")
    print(f"✅ Saved aligned + relabeled fused file to:\n{fused_out}")
    print("Final shape:", merged.shape)


# ==========================
# 4) Run for both fused files
# ==========================
align_and_replace_labels(FUSED_3_IN,  FUSED_3_OUT,  meta_all, key_cols)
align_and_replace_labels(FUSED_MOV_IN, FUSED_MOV_OUT, meta_all, key_cols)


Loading Data_ALL ...
Data_ALL shape: (1622677, 113)
Unique (subject,run,window_idx) in Data_ALL: 1622677

Processing fused file: C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_EMG_ECG_FIXED.csv
Shape (before): (5103389, 58)
Shape after inner-join with Data_ALL: (1622677, 60)
Number of rows where label_orig != label_all: 309761
✅ Saved aligned + relabeled fused file to:
C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_EMG_ECG_FINAL_FROM_DATA_ALL.csv
Final shape: (1622677, 58)

Processing fused file: C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_MOV_FIXED.csv
Shape (before): (5103389, 32)
Shape after inner-join with Data_ALL: (1622677, 34)
Number of rows where label_orig != label_all: 309761
✅ Saved aligned + relabeled fused file to:
C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_MOV_FINAL_FROM_DATA_ALL.csv
Final shape: (1622677, 32)


In [24]:
import pandas as pd

# ==========================
# 1) Paths
# ==========================
FUSED_3_PATH   = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_EMG_ECG_FINAL_FROM_DATA_ALL.csv"
FUSED_MOV_PATH = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_EEG_MOV_FINAL_FROM_DATA_ALL.csv"

OUT_MERGED_ALL = r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_ALL_FINAL_FROM_DATA_ALL.csv"

# ==========================
# 2) Load both fused files
# ==========================
print("Loading fused files ...")
df_eeg = pd.read_csv(FUSED_3_PATH)
df_mov = pd.read_csv(FUSED_MOV_PATH)

print("FUSED_EEG_EMG_ECG shape:", df_eeg.shape)
print("FUSED_EEG_MOV shape     :", df_mov.shape)

# Normalize label type
df_eeg["label"] = pd.to_numeric(df_eeg["label"], errors="coerce").fillna(0).astype(int)
df_mov["label"] = pd.to_numeric(df_mov["label"], errors="coerce").fillna(0).astype(int)

# Normalize keys
key_cols  = ["subject", "run", "window_idx"]
meta_cols = ["subject", "run", "window_idx", "label"]

for df, name in [(df_eeg, "EEG/EMG/ECG"), (df_mov, "MOV")]:
    df["subject"]    = df["subject"].astype(str).str.strip()
    df["run"]        = pd.to_numeric(df["run"], errors="raise").astype(int)
    df["window_idx"] = pd.to_numeric(df["window_idx"], errors="raise").astype(int)
    print(f"\n[{name}] unique (subject,run,window_idx):",
          df[key_cols].drop_duplicates().shape[0])

# ==========================
# 3) Label statistics for each file
# ==========================
def show_label_stats(df, title):
    print(f"\n===== LABEL STATS: {title} =====")
    counts = df["label"].value_counts().sort_index()
    print("Counts per label:")
    print(counts)

    perc = df["label"].value_counts(normalize=True).sort_index() * 100
    print("\nPercentages per label (%):")
    print(perc)

show_label_stats(df_eeg, "FUSED_EEG_EMG_ECG_FINAL_FROM_DATA_ALL")
show_label_stats(df_mov, "FUSED_EEG_MOV_FINAL_FROM_DATA_ALL")

# ==========================
# 4) Sanity check: keys alignment
# ==========================
keys_eeg = set(map(tuple, df_eeg[key_cols].values))
keys_mov = set(map(tuple, df_mov[key_cols].values))

print("\n===== KEY ALIGNMENT CHECK =====")
print("Unique keys in EEG/EMG/ECG:", len(keys_eeg))
print("Unique keys in MOV        :", len(keys_mov))
print("Intersection              :", len(keys_eeg & keys_mov))

if keys_eeg != keys_mov:
    print("⚠ WARNING: key sets are not identical. Merge will use INNER join on keys+label.")
else:
    print("✅ Key sets are identical.")

# ==========================
# 5) Build feature lists (avoid meta duplication)
# ==========================
eeg_feat_cols = [c for c in df_eeg.columns if c not in meta_cols]
mov_feat_cols = [c for c in df_mov.columns if c not in meta_cols]

print("\nNumber of EEG/EMG/ECG features:", len(eeg_feat_cols))
print("Number of MOV features        :", len(mov_feat_cols))

# Make sure there is no name clash between feature sets
overlap_feats = set(eeg_feat_cols) & set(mov_feat_cols)
if overlap_feats:
    print("\n⚠ WARNING: overlapping feature names between EEG and MOV:")
    print(overlap_feats)
    print("These columns will be duplicated with suffixes by pandas. "
          "Consider renaming them manually if needed.")

# ==========================
# 6) Merge all features into a single dataset
# ==========================
# We merge on meta columns so that:
#  - meta columns appear only once
#  - all features from both sides are included
df_eeg_sub = df_eeg[meta_cols + eeg_feat_cols].copy()
df_mov_sub = df_mov[meta_cols + mov_feat_cols].copy()

merged = df_eeg_sub.merge(
    df_mov_sub,
    on=meta_cols,   # merge on subject, run, window_idx, label
    how="inner",
    validate="1:1"
)

print("\n===== MERGED DATASET =====")
print("Merged shape:", merged.shape)

# Label stats for merged file
show_label_stats(merged, "MERGED ALL FEATURES (EEG+EMG+ECG+MOV)")

# ==========================
# 7) Save merged dataset
# ==========================
merged.to_csv(OUT_MERGED_ALL, index=False, encoding="utf-8-sig")
print("\n✅ Saved final merged dataset to:")
print(OUT_MERGED_ALL)


Loading fused files ...
FUSED_EEG_EMG_ECG shape: (1622677, 58)
FUSED_EEG_MOV shape     : (1622677, 32)

[EEG/EMG/ECG] unique (subject,run,window_idx): 1622677

[MOV] unique (subject,run,window_idx): 1622677

===== LABEL STATS: FUSED_EEG_EMG_ECG_FINAL_FROM_DATA_ALL =====
Counts per label:
label
0    1312916
1     289844
2      19917
Name: count, dtype: int64

Percentages per label (%):
label
0    80.910495
1    17.862088
2     1.227416
Name: proportion, dtype: float64

===== LABEL STATS: FUSED_EEG_MOV_FINAL_FROM_DATA_ALL =====
Counts per label:
label
0    1312916
1     289844
2      19917
Name: count, dtype: int64

Percentages per label (%):
label
0    80.910495
1    17.862088
2     1.227416
Name: proportion, dtype: float64

===== KEY ALIGNMENT CHECK =====
Unique keys in EEG/EMG/ECG: 1622677
Unique keys in MOV        : 1622677
Intersection              : 1622677
✅ Key sets are identical.

Number of EEG/EMG/ECG features: 54
Number of MOV features        : 28

===== MERGED DATASET =====
M

In [1]:
import pandas as pd

FUSED_4= r"C:\Users\LENOVO\Desktop\sen\sen\DATA_MARGED\FUSED_ALL_FINAL_FROM_DATA_ALL.csv"

# Load both
df4 = pd.read_csv(FUSED_4)

print("\n==============================")
print("FUSED EEG+EMG+ECG Columns:")
print(df4.columns.tolist())
print("Number of columns:", len(df4.columns))


FUSED EEG+EMG+ECG Columns:
['subject', 'run', 'window_idx', 'label', 'ch1_Delta Bandpower', 'ch1_Theta Bandpower', 'ch1_Alpha Bandpower', 'ch1_Beta Bandpower', 'ch1_Gamma Bandpower', 'ch1_Relative Delta Bandpower', 'ch1_Relative Theta Bandpower', 'ch1_Relative Alpha Bandpower', 'ch1_Relative Beta Bandpower', 'ch1_Relative Gamma Bandpower', 'ch1_Interquartile Range', 'ch1_Median', 'ch1_Entropy', 'ch1_Skewness', 'ch1_Kurtosis', 'ch1_Line Length', 'ch1_Hjorth Mobility', 'ch1_Hjorth Complexity', 'ch2_Delta Bandpower', 'ch2_Theta Bandpower', 'ch2_Alpha Bandpower', 'ch2_Beta Bandpower', 'ch2_Gamma Bandpower', 'ch2_Relative Delta Bandpower', 'ch2_Relative Theta Bandpower', 'ch2_Relative Alpha Bandpower', 'ch2_Relative Beta Bandpower', 'ch2_Relative Gamma Bandpower', 'ch2_Interquartile Range', 'ch2_Median', 'ch2_Entropy', 'ch2_Skewness', 'ch2_Kurtosis', 'ch2_Line Length', 'ch2_Hjorth Mobility', 'ch2_Hjorth Complexity', 'emg_RMS', 'emg_ZC', 'emg_SSC', 'emg_KURTOSIS', 'emg_SKEWNESS', 'emg_MEAN_